In [1]:
import pandas as pd
import numpy as np
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torch.nn as nn
import torchvision
import torch
# import time

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [2]:
num_classes = 2  # starfish and not starfish I guess

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained_backbone=False)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

model.load_state_dict(torch.load('/kaggle/input/wtcompetition/FastRCNN-AdamW.pt', map_location=device))
model.to(device)
model.eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=1e-05)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=1e-05)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=1e-05)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=1e-05)
          (relu

In [3]:
def slice_output(output: dict, confidence_threshold: float = 0.5) -> dict:
    '''
    this method is responsible for validating models output w.r.t confidence_threshold defined above. 
    It accepts an output dictionary from model, namely {'boxes':[], 'labels':[], 'scores':[]} 
    It returns a dictionary sliced to items with score above confidence_threshold
    '''

    num_valid_elements = np.sum(np.array(output['scores'].cpu()) >= confidence_threshold)
    # temporary option to make sure, that it returns at least one element, although it should probably be fixed, 
    # should there be any frames where there is no starfish
    if num_valid_elements == 0:
#         print('no valid elements')
        num_valid_elements = 1
    res = {}
    for key, value in output.items():
        res[key] = value[:num_valid_elements]
#     print('tmp_res {}'.format(res))
    return res

def format_for_submission(output:dict) -> str:
    '''
    this method is responsible for formating output so that it fits a submission format
    '''
    if len(output) == 0:
#         print('no prediction')
        return '0 0 0 0 0'
    output = slice_output(output)
    res_str = ''
    for bbox, score in zip(output['boxes'], output['scores']):
        bbox1, score1 = bbox.tolist(), score.tolist()
        w = int(bbox1[2] - bbox1[0])
        h = int(bbox1[3] - bbox1[1])
        x = int(bbox1[0])
        y = int(bbox1[1])
#         print(x, y, w, h)
        res_str += '{} {} '.format(score1, ' '.join(map(str, [x, y, w, h])))
    return res_str

# format_for_submission({'boxes': torch.Tensor([[205.0593, 148.9429, 242.3995, 171.5710],
#         [830.0285, 695.7241, 871.7217, 720.0000]]), 'labels': torch.Tensor([1, 1]), 'scores': torch.Tensor([0.6983, 0.2669])})

In [4]:
import sys
sys.path.append('/kaggle/input/tensorflow-great-barrier-reef')
import greatbarrierreef
env = greatbarrierreef.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission
with torch.no_grad():
    for (pixel_array, sample_prediction_df) in iter_test:
#         start_time = time.time()
        img = torch.from_numpy(pixel_array)
#         img = img.to(device)
        img = torch.permute(img, (2, 0, 1))
        min_image = img.min()
        max_image = img.max()
        # normalize image to 0-1 - required by torchvision
        img -= min_image
        img = torch.FloatTensor(img/max_image)
#         img = img.to(device)
        img = torch.unsqueeze(img, 0)
        img = img.to(device)
#         model.eval()
        output = model(img)
#         print('Models output: {}'.format(output))
#         output.to(torch.device('cpu'))
        result = format_for_submission(output[0])
        if result == '':
            result = '0 0 0 0 0'
#         print('Result: {}'.format(result))
        
        sample_prediction_df['annotations'] = result  # make your predictions here
        env.predict(sample_prediction_df)   # register your predictions
#         print("--- %s seconds ---" % (time.time() - start_time))
#     sample_prediction_df.to_csv('/kaggle/working/submission.csv')
sub_df = pd.read_csv('submission.csv')
sub_df

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


,index,annotations
0,0,0.26862549781799316 822 269 33 32
1,1,0.137962207198143 986 375 31 28
2,2,0.31031709909439087 988 382 33 28
